In [0]:
%pip install -U langgraph langsmith langchain transformers langchain_community

In [0]:
%pip install --upgrade mlflow

In [0]:
#list of availble models
import mlflow

client = mlflow.client.MlflowClient()
models = client.search_registered_models()

for model in models:
    print("Model Name:", model.name)
    if model.latest_versions is not None:
        for version in model.latest_versions:
            print(f" - Version: {version.version}, Stage: {version.current_stage}")

In [0]:
# Install torch in your Databricks notebook
%pip install torch


In [0]:
import mlflow

model_name = "system.ai.llama_v3_2_1b_instruct"
model_version = "2"

model_uri = f"models:/{model_name}/{model_version}"

print(f"Loading model: {model_name}...")
loaded_model = mlflow.pyfunc.load_model(model_uri)
print("Model successfully loaded!")

In [0]:
!python --version

In [0]:
# Define a simple prompt
prompt = "What are the key advantages of AI in modern businesses?"

In [0]:
# Prepare input in the required schema
input_data = {
    "messages": [
        {
            "role": "user",
            "content": "What are the key advantages of AI in modern businesses?"
        }
    ]
}

# Call the model's predict function
response = loaded_model.predict([input_data])

In [0]:
import json

print("Generated Response:")
print(json.dumps(response[0], indent=2))

In [0]:
from typing import Annotated, TypedDict, List
import mlflow
from langchain.schema import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# Define the State schema using Annotated for dynamic validation
class State(TypedDict):
    messages: Annotated[List[HumanMessage], add_messages]

# Define the StateGraph builder
graph_builder = StateGraph(State)

# Define the chatbot interaction function
def chatbot(state: State) -> State:
    """
    Processes the latest state, sends the input to the MLflow model, and
    directly appends the model's raw response to the state.
    """
    # Prepare input in the required schema (list of dictionaries)
    input_data = {
        "messages": [
            {
                "role": "user",  # Default role for HumanMessage
                "content": message.content
            }
            for message in state["messages"]
        ]
    }

    # Log the input being sent to the model (for debugging purposes)
    print("Input to MLflow model:", input_data)

    # Call the MLflow model's predict function
    response = loaded_model.predict([input_data])  # Pass as a list

    # Log the raw response from the model (for debugging purposes)
    print("Response from MLflow model:", response)

    # Append the raw response directly to the state
    # Convert the model's response to a single string (if necessary)
    ai_response = str(response)  # Ensure everything becomes a string representation

    # Update state by appending an AIMessage with the model's response
    return {
        "messages": state["messages"] + [AIMessage(content=ai_response)]
    }

# Add the chatbot node to the graph
graph_builder.add_node("chatbot", chatbot)

# Define the conversation flow by connecting nodes
graph_builder.add_edge(START, "chatbot")  # Start point to chatbot
graph_builder.add_edge("chatbot", END)   # Endpoint after chatbot response

# Compile the StateGraph
graph = graph_builder.compile()

# Example usage of the chatbot graph with initial state
if __name__ == "__main__":
    # Initialize a state with a single message
    state: State = {
        "messages": [
            HumanMessage(content="What are the key advantages of AI in modern businesses?")
        ]
    }

    # Execute the graph to process the conversation flow
    updated_state = graph.invoke(state)

    # Display the AI's response
    print("Chatbot response:", updated_state["messages"][-1].content)

In [0]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [0]:
from typing import Annotated, TypedDict, List
import mlflow
from langchain.schema import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# Define the State schema using Annotated
class State(TypedDict):
    messages: Annotated[List[HumanMessage], add_messages]
    
# Define the StateGraph builder
graph_builder = StateGraph(State)

# Define the chatbot interaction function
def chatbot(state: State) -> State:
    """
    Processes user input, sends it to the MLflow model, and appends 
    the model's response to the state.
    """
    # Prepare input for the model
    input_data = {
        "messages": [
            {
                "role": "user",
                "content": message.content
            }
            for message in state["messages"]
        ]
    }

    # Call the MLflow model's predict function
    response = loaded_model.predict([input_data])  # The predict method requires a list containing a dictionary
    ai_response = str(response)  # Convert the raw response to a string
    
    # Append the AI response to the state
    return {
        "messages": state["messages"] + [AIMessage(content=ai_response)]
    }


# Add the chatbot node to the graph
graph_builder.add_node("chatbot", chatbot)

# Define the conversation flow
graph_builder.add_edge(START, "chatbot")  # Start → chatbot
graph_builder.add_edge("chatbot", END)  # Chatbot → End

# Compile the graph
graph = graph_builder.compile()


# Function to stream graph updates
def stream_graph_updates(user_input: str):
    """
    Streams updates from the graph in response to user input.
    """
    # Prepare the input state with the user's message
    initial_state = {"messages": [{"role": "user", "content": user_input}]}
    
    # Stream events from the StateGraph
    for event in graph.stream(initial_state):
        for value in event.values():
            # Print the assistant's response, which is always the last message
            print("Assistant:", value["messages"][-1].content)


# Main loop for interaction
if __name__ == "__main__":
    print("Chatbot is ready! Type 'exit', 'quit', or 'q' to end the session.")
    while True:
        try:
            # Get user input
            user_input = input("User: ")
            # Exit condition
            if user_input.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break

            # Update the graph and stream responses
            stream_graph_updates(user_input)

        except Exception as e:
            # Fallback behavior for errors
            print(f"An error occurred: {e}")
            print("Defaulting to a static response...")
            fallback_input = "What do you know about LangGraph?"
            print("User:", fallback_input)
            stream_graph_updates(fallback_input)
            break